# Mask R-CNN Demo

## File modified by Casey Galvin to perform panoptic segmentation of Cityscapes validation dataset
## Modified section starts where noted below

A quick intro to using the pre-trained model to detect and segment objects.

In [2]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
#if not os.path.exists(COCO_MODEL_PATH):
#    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

Using TensorFlow backend.


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [3]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Create Model and Load Trained Weights

In [4]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [5]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Run Object Detection

In [6]:
from skimage.transform import rescale, resize
import matplotlib.pyplot as plt

## Pipeline to produce panoptic segmentations and calculate panoptic quality metric
### Instance segmentation performed using Mask R-CNN implementation (Matterport)
### Semantic segmentation files produced by Deeplab V3+ (Google)
#### Work below this line by Casey Galvin, building on Mask R-CNN implementation above

### Cityscapes label information taken from Deeplab V3+ semantic segmentation implementation

In [187]:
from __future__ import print_function, absolute_import, division
from PIL import Image

from collections import namedtuple





#--------------------------------------------------------------------------------

# Definitions

#--------------------------------------------------------------------------------



# a label and all meta information

Label = namedtuple( 'Label' , [



    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .

                    # We use them to uniquely name a class



    'id'          , # An integer ID that is associated with this label.

                    # The IDs are used to represent the label in ground truth images

                    # An ID of -1 means that this label does not have an ID and thus

                    # is ignored when creating ground truth images (e.g. license plate).

                    # Do not modify these IDs, since exactly these IDs are expected by the

                    # evaluation server.



    'trainId'     , # Feel free to modify these IDs as suitable for your method. Then create

                    # ground truth images with train IDs, using the tools provided in the

                    # 'preparation' folder. However, make sure to validate or submit results

                    # to our evaluation server using the regular IDs above!

                    # For trainIds, multiple labels might have the same ID. Then, these labels

                    # are mapped to the same class in the ground truth images. For the inverse

                    # mapping, we use the label that is defined first in the list below.

                    # For example, mapping all void-type classes to the same ID in training,

                    # might make sense for some approaches.

                    # Max value is 255!



    'category'    , # The name of the category that this label belongs to



    'categoryId'  , # The ID of this category. Used to create ground truth images

                    # on category level.



    'hasInstances', # Whether this label distinguishes between single instances or not



    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored

                    # during evaluations or not



    'color'       , # The color of this label

    ] )





#--------------------------------------------------------------------------------

# A list of all labels

#--------------------------------------------------------------------------------



# Please adapt the train IDs as appropriate for your approach.

# Note that you might want to ignore labels with ID 255 during training.

# Further note that the current train IDs are only a suggestion. You can use whatever you like.

# Make sure to provide your results using the original IDs and not the training IDs.

# Note that many IDs are ignored in evaluation and thus you never need to predict these!



labels = [

    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color

    Label(  'unlabeled'            ,  0 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),

    Label(  'ego vehicle'          ,  1 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),

    Label(  'rectification border' ,  2 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),

    Label(  'out of roi'           ,  3 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),

    Label(  'static'               ,  4 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),

    Label(  'dynamic'              ,  5 ,      255 , 'void'            , 0       , False        , True         , (111, 74,  0) ),

    Label(  'ground'               ,  6 ,      255 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),

    Label(  'road'                 ,  7 ,        0 , 'flat'            , 1       , False        , False        , (128, 64,128) ),

    Label(  'sidewalk'             ,  8 ,        1 , 'flat'            , 1       , False        , False        , (244, 35,232) ),

    Label(  'parking'              ,  9 ,      255 , 'flat'            , 1       , False        , True         , (250,170,160) ),

    Label(  'rail track'           , 10 ,      255 , 'flat'            , 1       , False        , True         , (230,150,140) ),

    Label(  'building'             , 11 ,        2 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),

    Label(  'wall'                 , 12 ,        3 , 'construction'    , 2       , False        , False        , (102,102,156) ),

    Label(  'fence'                , 13 ,        4 , 'construction'    , 2       , False        , False        , (190,153,153) ),

    Label(  'guard rail'           , 14 ,      255 , 'construction'    , 2       , False        , True         , (180,165,180) ),

    Label(  'bridge'               , 15 ,      255 , 'construction'    , 2       , False        , True         , (150,100,100) ),

    Label(  'tunnel'               , 16 ,      255 , 'construction'    , 2       , False        , True         , (150,120, 90) ),

    Label(  'pole'                 , 17 ,        5 , 'object'          , 3       , False        , False        , (153,153,153) ),

    Label(  'polegroup'            , 18 ,      255 , 'object'          , 3       , False        , True         , (153,153,153) ),

    Label(  'traffic light'        , 19 ,        6 , 'object'          , 3       , False        , False        , (250,170, 30) ),

    Label(  'traffic sign'         , 20 ,        7 , 'object'          , 3       , False        , False        , (220,220,  0) ),

    Label(  'vegetation'           , 21 ,        8 , 'nature'          , 4       , False        , False        , (107,142, 35) ),

    Label(  'terrain'              , 22 ,        9 , 'nature'          , 4       , False        , False        , (152,251,152) ),

    Label(  'sky'                  , 23 ,       10 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),

    Label(  'person'               , 24 ,       11 , 'human'           , 6       , True         , False        , (220, 20, 60) ),

    Label(  'rider'                , 25 ,       12 , 'human'           , 6       , True         , False        , (255,  0,  0) ),

    Label(  'car'                  , 26 ,       13 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),

    Label(  'truck'                , 27 ,       14 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),

    Label(  'bus'                  , 28 ,       15 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),

    Label(  'caravan'              , 29 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),

    Label(  'trailer'              , 30 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),

    Label(  'train'                , 31 ,       16 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),

    Label(  'motorcycle'           , 32 ,       17 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),

    Label(  'bicycle'              , 33 ,       18 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),

    Label(  'license plate'        , -1 ,       -1 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),

]





#--------------------------------------------------------------------------------

# Create dictionaries for a fast lookup

#--------------------------------------------------------------------------------



# Please refer to the main method below for example usages!



# name to label object

name2label      = { label.name    : label for label in labels           }

# id to label object

id2label        = { label.id      : label for label in labels           }

# trainId to label object

trainId2label   = { label.trainId : label for label in reversed(labels) }

# category to list of label objects

category2labels = {}

for label in labels:

    category = label.category

    if category in category2labels:

        category2labels[category].append(label)

    else:

        category2labels[category] = [label]



#--------------------------------------------------------------------------------

# Assure single instance name

#--------------------------------------------------------------------------------



# returns the label name that describes a single instance (if possible)

# e.g.     input     |   output

#        ----------------------

#          car       |   car

#          cargroup  |   car

#          foo       |   None

#          foogroup  |   None

#          skygroup  |   None

def assureSingleInstanceName( name ):

    # if the name is known, it is not a group

    if name in name2label:

        return name

    # test if the name actually denotes a group

    if not name.endswith("group"):

        return None

    # remove group

    name = name[:-len("group")]

    # test if the new name exists

    if not name in name2label:

        return None

    # test if the new name denotes a label that actually has instances

    if not name2label[name].hasInstances:

        return None

    # all good then

    return name

In [188]:
######
# 
# Computes IoU between instance and semantic segmentations
# Used to determine if classification matches between segmantations
# 
# Arguments:
# ins_results is results array from Mask R-CNN instance segmentation (r variable output in instance segmentation model)
# ins is specific instance being analyzed
# sem_seg is semantic segmentation result from Deeplab
# city_id is Cityscapes ID for object corresponding to ins
#
# Returns:
# IoU for object corresponding to ins in ins_results
#
######

def ins_sem_compare(ins_results, ins, sem_seg, city_id):
    
    #define roi for instance and semantic comparison
    roi = ins_results['rois'][ins]
    
    y1 = roi[0]
    x1 = roi[1]
    y2 = roi[2]
    x2 = roi[3]  
    
    #Cast sem_seg to binary mask
    sem_seg_binary = sem_seg[y1:y2,x1:x2] == city_id
    
    #find intersection between instance and semantic segmentation
    #only looks in instance ROI
    intersection_map = np.logical_and(r['masks'][:, :, ins][y1:y2,x1:x2], sem_seg_binary)
    
    intersection = np.sum(intersection_map)
    
    #find union, including subtracting intersection to avoid double counting
    union = np.sum(ins_results['masks'][:,:,ins]) + np.sum(sem_seg_binary) - intersection
    
    #calculate intersection over union
    iou = intersection/union
    
    return iou

In [189]:
######
# 
# Converts COCO ids (MASK R-CNN objects in Matterport implementation) to Cityscapes ids (Deeplab semantic segmentation)
# Needed to match output of segmentation from the two outputs
#
# Arguments:
# coco_id is ID of an object using the COCO dataset nomenclature
# 
# Returns:
# city_id is ID of input object using the Cityscapes nomenclature
#
######

def coco2city(coco_id):
    object_class = class_names[coco_id]
    city_id = name2label[object_class].id
    return city_id

In [219]:
######
#
# Produces colors for panoptic segmentation map
# Shades different instances of the same class a different color
#
# Arguments:
# pan_seg is panoptic segmentation with object labels
#
# Returns:
# colored_pan_seg is colored panoptic segmentation map
# cmap is color map information to plot or save using Matplotlib
#
######

def pan_colors(pan_seg):
    
    import colorsys
    import matplotlib.colors

    colors = []

    for pan_id in np.unique(pan_seg):

        if pan_id > 999: #multiplication factor to create enumerated instances

            classId = int(pan_id/1000)
            objectint = pan_id - classId*1000

            labelcolor = id2label[classId].color

            hls_color = colorsys.rgb_to_hls(labelcolor[0], labelcolor[1], labelcolor[2])
            newcolor = [hls_color[0], (1 - objectint/len(np.unique(pan_seg))) * hls_color[1], hls_color[2]]

            color = colorsys.hls_to_rgb(newcolor[0], newcolor[1], newcolor[2])

        else:
            color = id2label[pan_id].color

        norm_color = [need2norm/255 for need2norm in color]

        colors.append(norm_color)

    cmap = matplotlib.colors.ListedColormap(colors)

    norm = matplotlib.colors.BoundaryNorm(np.arange(len(np.unique(pan_seg))+1)-0.5, len(colors))
    
    u, ind = np.unique(pan_seg, return_inverse=True)
    
    colored_pan_seg = np.reshape(ind, pan_seg.shape)
    
    return colored_pan_seg, cmap
        
    #plt.figure(figsize=(15,30))
    #plt.imshow(b, cmap=cmap, norm=norm, shape=[1024, 2048])
    #plt.axis("off")

In [220]:
######
#
# Function that combines semantic segmentation and instance segmentations using heuristics
# 
# Arguments:
# ins_results is results array from Mask R-CNN instance segmentation (r variable output in instance segmentation model)
# sem_seg is semantic segmentation result from Deeplab
#
# Returns:
# pan_map is panoptic segmentation output with class labels (Cityscapes nomenclature)
# pan_map_color is panoptic segmentation output colored according to pan_colors function
# cmap is color map produced by pan_colors function used for plotting and saving pan_map_color with Matplotlib functions
#
######

def create_pan_seg(ins_results, sem_seg):
    #This follows the heuristic of the panoptic segmentation paper
    
    #Create a copy of the semantic segmentation to overwrite instances
    #Casts sem_seg into int32 to accommodate enumerated class_ids
    pan_map = sem_seg.astype(int).copy()
    
    #Stores coco2city translation and class_id enumerations
    inst_dict = {}
    
    #List of class_ids to mark void for remaining semantic labels
    markvoid = []
    
    #For each instance in the instance segmentations...
    for ins, obj_id in enumerate(ins_results['class_ids']):
        
        #Possible improvement:
        #If instance segmentation confidence is low, don't use
        
        #Translate COCO to Cityscapes class_ids for current object
        try:
            city_id = coco2city(obj_id)
        except KeyError:
            print("Found an object I am not trained to handle:", obj_id)
            continue
            
        
        #Compare instance and semantic prediction
        #If results do not agree, mark void
        iou = ins_sem_compare(ins_results, ins, sem_seg, city_id)
        
        if iou < 0.5:
            
            #Mark pixels as void
            pan_map[np.nonzero(ins_results['masks'][:,:,ins])] = 0
            
        else:
            
            #Get current enumeration of object
            inst_int = inst_dict.get(city_id, 0)
            
            #Transform city_id to instance_id --> Follows Cityscapes convention of 5 digit number: first 2 digits are class_id, last 3 are enumerated instance
            inst_id = city_id*1000 + inst_int
            
            #Enumerate city_id to the next instance
            inst_dict[city_id] = inst_int + 1
            
            #Append ID to mark void for remaining semantic pixels
            if inst_int == 0:
                markvoid.append(city_id)

            #Write instance ID onto panoptic map
            pan_map[np.nonzero(ins_results['masks'][:,:,ins])] = inst_id
    
    #Void remaining semantic pixels for enumerated objects
    for void in markvoid:
        pan_map[pan_map == void] = 0
    
    #Void remaining semantic pixels that fall below a critical abundance threshold
    #Define critical area factor --> can be adjusted
    void_factor = 1/pan_map.shape[1]
    
    for void in np.unique(pan_map):
        
        #Compare size of IDs and mark void if smaller than void_factor
        if np.sum(pan_map == void)/pan_map.size < void_factor:
            pan_map[pan_map == void] = 0
    
    #plt.imshow(pan_map, cmap='hot')
    pan_map_color, cmap = pan_colors(pan_map)
        
    return pan_map, pan_map_color, cmap

In [210]:
######
#
# Function to calculate IoU between panoptic segmentation and ground truth files
# 
# Arguments:
# pan_seg is panoptic segmentation with object labels
# ground_truth is ground truth file from Cityscapes dataset (instance segmentation, using XXYYY format, where XX is label and YYY is enumerated instance)
# class_id is Cityscapes class ID from pan_seg
# matched_id is instance segmentation ID that has been matched with class ID (XX is same between images, however YYY can be different for same instance)
#
# Returns:
# iou is intersection over union for pan_seg and ground_truth
#
######

def calc_iou(pan_seg, ground_truth, class_id, matched_id):
    
    intersection = np.sum((pan_seg == class_id) & (ground_truth == matched_id))
    
    union = np.sum((pan_seg == class_id) | (ground_truth == matched_id))
    
    iou = intersection/union
    
    return iou

######
#
# Function to calculate panoptic quality metric (and segmentation and recognition qualities) between panoptic segmentation and ground truth files
# 
# Arguments:
# pan_seg is panoptic segmentation with object labels
# ground_truth is ground truth file from Cityscapes dataset (instance segmentation, using XXYYY format, where XX is label and YYY is enumerated instance)
#
# Returns:
# seg_qual is segment quality (average IoU over true positives)
# rec_qual is recognition quality (F1 score)
# pan_qual is panoptic quality (seg_qual * rec_qual)
#
######

def calc_pq(pan_seg, ground_truth):
    
    from scipy import stats 
    
    iou_match = []
    fp_ids = []
    fn_ids = []
    results_list = []
    
    for class_id in np.unique(pan_seg):

        if class_id == 0:
            continue

        if class_id > 999: #factor for enumeration

            #match likely instance IDs based on mode over class ID in overlapping pixels
            matched_id = stats.mode(ground_truth[pan_seg == class_id], axis=None)[0][0]
            occurs = stats.mode(ground_truth[pan_seg == class_id], axis=None)[1][0]

        else:

            #For stuff IDs, class_id and matched_id will be equal (no individual instances)
            matched_id = class_id

        iou = calc_iou(pan_seg, ground_truth, class_id, matched_id)

        #print(class_id, iou)
        
        if iou < 0.5:
            fp_ids.append(class_id)
            
            if class_id in np.unique(ground_truth):
                fn_ids.append(class_id)
                
            continue

        iou_match.append(iou)


    tp = len(iou_match)
    fp = len(fp_ids)
    fn = len(fn_ids)
        
    seg_qual = np.sum(iou_match)/len(iou_match)
    
    rec_qual = tp/(tp + 0.5*fp + 0.5*fn)
    
    pan_qual = seg_qual * rec_qual
    
    #print("sq =", seg_qual, "rq =", rec_qual)
    #print("pq =", pan_qual)
    #print("True positives:", tp)
    #print("False positives:", fp)
    #print("False negatives:", fn)
    
    return seg_qual, rec_qual, pan_qual

## Pipeline to analyze all images from Cityscapes validation set and save necessary information

In [223]:
#Pipeline to analyze all images and save necessary information

from os import listdir
from os.path import isfile, join

#Load ground truth image file names
val_dir = r"C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\cityscapes\gtFine\val"
cities = os.listdir(val_dir)

gt_files = []
for city in cities:
    gt_files.extend([city + "\\" + f for f in listdir(val_dir + "\\" + city) if "instanceIds" in f])

print("Loaded ground truth file locations...")
    
#Semantic segmentation image locations
img_dir = r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\cityscapes\exp\train_on_train_set\vis\segmentation_results'
raw_dir = r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\cityscapes\exp\train_on_train_set\vis\raw_segmentation_results'

sem_str = '_prediction.png'
raw_str = '_raw.png'
image_str = '_image.png'

print("Loaded semantic segmentation file locations...")

#Directory for saving panoptic segmentation results
pan_dir = r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\cityscapes\exp\train_on_train_set\vis\panoptic_results'
pan_str = '_panoptic.png'

#Results lists
sq = []
rq = []
pq = []

print("Starting panoptic segmentation routine...")
#Create panoptic segmentation and return panoptic quality score for images
for i in range(0, len(gt_files)):
    num_str = str(i).zfill(6)
    
    sem_file = '\\' + num_str + sem_str
    image_file = '\\' + num_str + image_str
    raw_file = '\\' + num_str + raw_str
    
    image = skimage.io.imread(img_dir + image_file)

    # Run detection
    results = model.detect([image], verbose=0)
    r = results[0]
    print('Processed image', num_str)
    
    # Loading semantic segmentation images
    sem_img = Image.open(img_dir + sem_file)
    sem_array = np.array(sem_img)
    raw_sem = Image.open(raw_dir + raw_file)
    raw_sem_array = np.array(raw_sem)
    
    # Create panoptic segmentation from semantic and object segmentations using heuristics
    pan_seg, pan_seg_color, cmap = create_pan_seg(r, raw_sem_array)
    print("Made panoptic segmentation for image", num_str)
    
    #Ground truth images
    gt_image = Image.open(val_dir + "\\" + gt_files[i])
    gt_array = np.array(gt_image)
    
    # Analyze panoptic segmentation relative to ground truth (pan_city_ins_arr)
    seg_qual, rec_qual, pan_qual = calc_pq(pan_seg, gt_array)
    
    # Append results to master lists for analysis
    sq.append(seg_qual)
    rq.append(rec_qual)
    pq.append(pan_qual)
    
    # Save resulting panoptic segmentation images with indexed colors
    pan_file = '\\' + num_str + pan_str
    plt.imsave(pan_dir + pan_file, pan_seg_color, cmap=cmap)
    
    print("Analyzed and saved panoptic segmentation for image", num_str)

Loaded ground truth file locations...
Loaded semantic segmentation file locations...
Starting panoptic segmentation routine...
Processed image 000000
Made panoptic segmentation for image 000000
Analyzed and saved panoptic segmentation for image 000000
Processed image 000001
Made panoptic segmentation for image 000001
Analyzed and saved panoptic segmentation for image 000001
Processed image 000002
Found an object I am not trained to handle: 27
Made panoptic segmentation for image 000002
Analyzed and saved panoptic segmentation for image 000002
Processed image 000003
Found an object I am not trained to handle: 39
Made panoptic segmentation for image 000003
Analyzed and saved panoptic segmentation for image 000003
Processed image 000004
Made panoptic segmentation for image 000004
Analyzed and saved panoptic segmentation for image 000004
Processed image 000005
Made panoptic segmentation for image 000005
Analyzed and saved panoptic segmentation for image 000005
Processed image 000006
Made p

## Calculate average values of panoptic, segmentation and recognition quality metrics

In [224]:
print(np.average(pq), np.average(sq), np.average(rq))

0.580351830122578 0.7875140499247384 0.7372931654369143
